In [1]:
import pandas as pd
import numpy as np
from matplotlib import colors
from matplotlib import rc
import matplotlib.pyplot as plt

from matplotlib import rcParams

latex_path = '/usr/local/texlive/2023/bin/universal-darwin'

rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = r'\usepackage{{amsmath}}\usepackage{{amsfonts}}\usepackage{{amssymb}}'
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Computer Modern Roman']
rcParams['mathtext.fontset'] = 'cm'

In [2]:

file_name = 'CCHU_NIRS.xlsx'
df = pd.read_excel(file_name)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
df = df[['Subject_ID','probe', 'Block1 (S1)' ,'Filter Used for Block 1 (S1)','Block2 (S1)','Filter Used for Block 2 (S1)','Block1 (S2)','Filter Used for Block 1 (S2)', 'Block2 (S2)',
                'Filter Used for Block 2 (S2)', 'Block1 (S3)','Filter Used for Block 1 (S3)', 'Block2 (S3)','Filter Used for Block 2 (S3)']]

In [4]:
df

,Subject_ID,probe,Block1 (S1),Filter Used for Block 1 (S1),Block2 (S1),Filter Used for Block 2 (S1),Block1 (S2),Filter Used for Block 1 (S2),Block2 (S2),Filter Used for Block 2 (S2),Block1 (S3),Filter Used for Block 1 (S3),Block2 (S3),Filter Used for Block 2 (S3)
0,NC301,CCHUNB6A,1084.0,0.3,2103.0,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NC301,CCHUNB6A,1084,0.5,2103,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NC302,CCHUNB6A,1084,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NC302,CCHUNB6A,1084,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NC303,CCHUNB6A,1084,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2343,NE300,CCHUNB6A,1084,0.2,2103,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2344,NE300,CCHUNB6A,1084,0.3,2103,0.3,1084,0.3,2103,0.4,NaN,NaN,NaN,NaN
2345,NE300,CCHUNB6A,1084,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2346,NE300,CCHUNB6A,1084,0.3 Good for A & B; No Filter Good for C & D,2103,0.3 Good for A & B; No Filter Good for C & D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# Function for identifying how many rows have the D distance and extracting that value, filtering A, B and C distances. 

def extracting_and_filtering(df, block, column):
    df_block = (df[df['Block1 (S1)']== block])
    df_block_f = df_block.copy(); index_of_row = df_block_f.index.tolist()
    rejected_elements = []
    found_number = False
    for i,f in zip(index_of_row,df_block_f[column]): 
        try:
            if any(letter in f for letter in ['A', 'B', 'C','D']):

                if 'D' in f:
                    found_number = False
                    try: 
                        index = f.index(' D')
                    except:
                        try: 
                            index = f.index('CD')
                        except: 
                            index = f.index(',D')
                        
                    while found_number == False:
                        if any(char in f[index-3:index] for char in ['No', 'NO', 'no',]):
                            found_number = True
                            df_block_f[column][i] = np.nan
                            rejected_elements = np.append(rejected_elements,str(i) +' '+f)
                        elif any(number in f[index-3:index] for number in ['1.','0.']):
                            found_number = True
                            #print(fr'Element {i}  :  {f} -- extracted value:  {f[index-3:index]}')
                            df_block_f[column][i] = float(f[index-3:index])
                        else: index = index - 1 
                    
                else: 
                    rejected_elements = np.append(rejected_elements,str(i) +' '+f)
                    df_block_f[column][i] = np.nan

            elif ';' in f:
                df_block_f[column][i] = np.nan
                rejected_elements = np.append(rejected_elements,str(i) +' '+f)


        except: 
            pass
    
    df_block_f = df_block_f.dropna(subset=[column])
    return df_block_f, rejected_elements



# Block 1 Sesion 1

In [55]:
print('Different block names for Block 1 (S1)')
print(df['Block1 (S1)'].unique())
df.loc[df['Block1 (S1)'] == '1084', 'Block1 (S1)'] = float(1084)
df.loc[df['Block1 (S1)'] == '2101', 'Block1 (S1)'] = float(2101)
df.loc[df['Block1 (S1)'] == '2102', 'Block1 (S1)'] = float(2102)
df = df[df['Block1 (S1)'] != '-']; df = df[df['Block1 (S1)'] != '-8'];
print('')
print('Condensed Block Names for Block 1 (S1)')
print(df['Block1 (S1)'].unique())


Different block names for Block 1 (S1)
[1084.0 nan 2101.0 2102.0 2103.0]

Condensed Block Names for Block 1 (S1)
[1084.0 nan 2101.0 2102.0 2103.0]


In [32]:
for b in df['Block1 (S1)'].unique():
    print('Block name '+str(b) + ' has '+str(len(df[df['Block1 (S1)']==b])) + ' rows')


Block name 1084.0 has 2282 rows
Block name nan has 0 rows
Block name 2101.0 has 29 rows
Block name 2102.0 has 8 rows
Block name 2103.0 has 1 rows


## 1084

In [66]:
df_b1_1084_f, rejected_elements = extracting_and_filtering(df,1084,'Filter Used for Block 1 (S1)')

In [63]:
#pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', None)

In [36]:
print(fr"{len(rejected_elements)} rows were filtered for the block 1084")

42 rows were filtered for the block 1084


In [37]:
df_b1_1084_f

,Subject_ID,probe,Block1 (S1),Filter Used for Block 1 (S1),Block2 (S1),Filter Used for Block 2 (S1),Block1 (S2),Filter Used for Block 1 (S2),Block2 (S2),Filter Used for Block 2 (S2),Block1 (S3),Filter Used for Block 1 (S3),Block2 (S3),Filter Used for Block 2 (S3)
0,NC301,CCHUNB6A,1084.0,0.3,2103.0,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NC301,CCHUNB6A,1084.0,0.5,2103,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NC302,CCHUNB6A,1084.0,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NC302,CCHUNB6A,1084.0,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NC303,CCHUNB6A,1084.0,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2342,NE300,CCHUNB6A,1084.0,0.5,2103,0.6,1084,0.9; 0.6,2103,1.0; 0.8,NaN,NaN,NaN,NaN
2343,NE300,CCHUNB6A,1084.0,0.2,2103,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2344,NE300,CCHUNB6A,1084.0,0.3,2103,0.3,1084,0.3,2103,0.4,NaN,NaN,NaN,NaN
2345,NE300,CCHUNB6A,1084.0,0.3,2103,0.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Manual Processing of Data for 1084

In [38]:
index_of_row = df_b1_1084_f.index.tolist()

for i in index_of_row: 
    try:
        if any(char in df_b1_1084_f['Filter Used for Block 1 (S1)'][i] for char in ['nd', '/','No Filter','No filter', ',','..',"'0.4'"]):
            print(fr"index {i}: {df_b1_1084_f['Filter Used for Block 1 (S1)'][i]}")
    except:
        pass

    


index 78: 0.4 nd
index 829: 0.3 for first block msmt,0.2 for second
index 834: 1.1/0.7
index 973: No filter
index 1100: No filters noted
index 1496: 0.7, 0.4
index 1518: 0.4 nd
index 1530: 0..9
index 1781: No Filter
index 1800: No Filter
index 1959: 0.7, 0.3
index 1978: 0.3 nd
index 2269: 0.2 nd


In [39]:
df_b1_1084_f['Filter Used for Block 1 (S1)'][78] = 0.4
df_b1_1084_f['Filter Used for Block 1 (S1)'][829] = 0.3
df_b1_1084_f['Filter Used for Block 1 (S1)'][1518] = 0.3
df_b1_1084_f['Filter Used for Block 1 (S1)'][1530] = 0.9
df_b1_1084_f['Filter Used for Block 1 (S1)'][1978] = 0.3
df_b1_1084_f['Filter Used for Block 1 (S1)'][2269] = 0.2
df_b1_1084_f['Filter Used for Block 1 (S1)'][2083] = 0.4

#Deleting Values
df_b1_1084_f = df_b1_1084_f.drop(1959)
df_b1_1084_f = df_b1_1084_f.drop(1800)
df_b1_1084_f = df_b1_1084_f.drop(1781)
df_b1_1084_f = df_b1_1084_f.drop(1496)
df_b1_1084_f = df_b1_1084_f.drop(1100)
df_b1_1084_f = df_b1_1084_f.drop(973)
df_b1_1084_f = df_b1_1084_f.drop(834)

index_of_row = df_b1_1084_f.index.tolist()

In [40]:
# Checking if all of values are numbers
errors = False
for i in index_of_row: 
    try:
        float(df_b1_1084_f['Filter Used for Block 1 (S1)'][i])
    except:
        print(fr"index {i} can't be converted to float its value is {df_b1_1084_f['Filter Used for Block 1 (S1)'][i]}")
        errors = True

if errors: pass
else: 
    print('All of the values are numbers and they are now in float format for 1084')
    df_b1_1084_f['Filter Used for Block 1 (S1)'] = df_b1_1084_f['Filter Used for Block 1 (S1)'].astype(float)


All of the values are numbers and they are now in float format for 1084


## 2101

In [67]:
df_b1_2101_f, rejected_elements_2101 = extracting_and_filtering(df,2101,'Filter Used for Block 1 (S1)')

In [68]:
print(fr"{len(rejected_elements_2101)} elements were filtered from the 2101 block")

2 elements were filtered from the 2101 block


In [58]:
index_of_row = df_b1_2101_f.index.tolist()

for i in index_of_row: 
    try:
        if any(char in df_b1_2101_f['Filter Used for Block 1 (S1)'][i] for char in ['nd', '/','No Filter','No filter', ',','..',"'0.4'"]):
            print(fr"index {i}: {df_b1_2101_f['Filter Used for Block 1 (S1)'][i]}")
    except:
        pass


In [61]:
errors = False
for i in index_of_row: 
    try:
        float(df_b1_2101_f['Filter Used for Block 1 (S1)'][i])
    except:
        print(fr"index {i} can't be converted to float its value is {df_b1_2101_f['Filter Used for Block 1 (S1)'][i]}")
        errors = True

if errors: pass
else: 
    print('All of the values are numbers and they are now in float format for 2101')
    df_b1_2101_f['Filter Used for Block 1 (S1)'] = df_b1_2101_f['Filter Used for Block 1 (S1)'].astype(float)

All of the values are numbers and they are now in float format for 2101
